#### Build a simple LLM application with LCEL

Use: The application will translate the text from english to another language.

In [9]:
import os 
from dotenv import load_dotenv
load_dotenv

import openai

open.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key 


In [ ]:
from langchain_groq import ChatGroq 
from langchain_openai import ChatOpenAI

# Reload the .env file to ensure the groq_api_key is set
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

# Ensure the groq_api_key is set correctly
if not groq_api_key:
	raise ValueError("groq_api_key is not set. Please check your .env file or environment variables.")

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DAD3089BB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DAD308ABD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content = "Translate the following from English to French"),
    HumanMessage(content = "Hello, How are you")

]

result = model.invoke(messages)

In [16]:
result 

AIMessage(content='Bonjour, Comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 21, 'total_tokens': 32, 'completion_time': 0.02, 'prompt_time': 0.002149745, 'queue_time': 0.23396525499999998, 'total_time': 0.022149745}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-83802fdd-d92a-4d08-81ff-e429ab5d1937-0', usage_metadata={'input_tokens': 21, 'output_tokens': 11, 'total_tokens': 32})

In [18]:
# using String output parser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)



'Bonjour, Comment allez-vous ? \n'

In [20]:
# using LCEL - chain the component 

chain = model | parser
chain.invoke(messages)

'Bonjour, Comment allez-vous ?  \n'

In [28]:
## prompt templates 

from langchain_core.prompts import ChatPromptTemplate 

generic_template = "Translate the following into {Language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),("user", "{text}")]

)

In [30]:
prompt.invoke({"Language": "French", "text": "Hello"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [32]:
# Convert the AIMessage object to a list of messages
[SystemMessage(content="Translate the following from English to French"), result]

[SystemMessage(content='Translate the following from English to French', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Bonjour, Comment allez-vous ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 21, 'total_tokens': 32, 'completion_time': 0.02, 'prompt_time': 0.002149745, 'queue_time': 0.23396525499999998, 'total_time': 0.022149745}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-83802fdd-d92a-4d08-81ff-e429ab5d1937-0', usage_metadata={'input_tokens': 21, 'output_tokens': 11, 'total_tokens': 32})]

In [34]:
chain = prompt | model | parser
chain.invoke({"Language": "French", "text": "Hello"})



'Bonjour \n'

In [40]:
# Chaining together components with LCEL 
chain = prompt | model | parser 
chain.invoke({"Language": "French", "text": "Hello"})


"Bonjour \n\n\nLet me know if you have any other phrases you'd like translated!\n"